In [17]:
import numpy as np
import pandas as pd

class Multinomial_NB:
    
    def __init__(self, alpha=1.0):
        self.alpha = alpha
    
    def fit(self, X, Y):
        mu_jc = np.array([])
        
        for j in range(len(Y)):
            document_count = zip(np.unique(Y, return_counts))
            sum = np.sum(X[:, j]*document)
            for i in range(len(X)):
                sum += X[i][j]
        
    
    def predict(self, X):
        pass
    
    def predict_proba(self, X):
        pass
    
    def predict_log_proba(self, X):
        pass

In [24]:
import pandas as pd
df = pd.read_csv("./datasets/SMSSpamCollection.csv",header=None,names=["label","text"])
import nltk
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
df['text_lemmatized'] = df['text'].map(lambda text: ' '.join(lemmatizer.lemmatize(w) for w in nltk.word_tokenize(text)))
y=np.array(df['label'])
unique, counts = np.unique(y, return_counts=True)
y_counts=dict(zip(unique, counts))


